In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow import keras
from sklearn.model_selection import train_test_split

print(tf.__version__)
print(keras.__version__)

1.12.0
2.1.6-tf


In [2]:
tf.test.is_gpu_available()

True

In [3]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
import tensorflow.keras.applications.resnet50 as resnet50 
import tensorflow.keras.applications.vgg19 as vgg19
# import tensorflow.keras.applications.vgg19 as vgg19
import tensorflow.keras.applications.densenet as densenet
import tensorflow.keras.applications.inception_v3 as inception_v3

In [6]:
import tensorflow.keras.applications.vgg19 as vgg19

In [7]:
import tensorflow.keras.preprocessing.image as image
import tensorflow.keras.losses as losses
import tensorflow.keras.layers as layers
import tensorflow.keras.models as models
import tensorflow.keras.callbacks as callbacks
import tensorflow.keras.backend as K

In [9]:
from scipy.io import loadmat
from os.path import join

prefix = './Stanford_Dog_Breed'
imgfix = 'Images'

In [31]:
ImgTrainGen = image.ImageDataGenerator(preprocessing_function=vgg19.preprocess_input, 
                                  width_shift_range=0.2, 
                                  height_shift_range=0.2, 
                                  shear_range=0.2, 
                                  zoom_range=0.2,
                                  rotation_range=0.2,
                                  fill_mode='nearest',
                                  horizontal_flip=True, 
                                  vertical_flip=True, 
#                                   validation_split=1/6.
                                      )

Train = ImgTrainGen.flow_from_directory(join(prefix, imgfix, 'Train'), 
                                   target_size=(224, 224), 
                                   class_mode='sparse', 
                                   batch_size=32, 
                                   shuffle=True, 
                                   seed=None, 
                                   subset='training', 
                                   interpolation='nearest')

ImgTestGen = image.ImageDataGenerator(preprocessing_function=vgg19.preprocess_input, validation_split=1/6.)

Test  = ImgTestGen.flow_from_directory(join(prefix, imgfix, 'Test' ), 
                                       target_size=(224, 224), 
                                       class_mode='sparse', 
                                       batch_size=32, 
                                       shuffle=False, 
                                       interpolation='nearest')

Found 12000 images belonging to 120 classes.
Found 8580 images belonging to 120 classes.


In [9]:
def MultiTaskDataGen(ImgGen):
    while True:
        X = ImgGen.next()
        yield [X[0], X[1]], [X[1], X[1], X[1]]
        
TrainData = MultiTaskDataGen(Train)
# ValidData = MultiTaskDataGen(Valid)
TestData  = MultiTaskDataGen(Test)

# VGG 19

In [12]:
vgg19_naive = vgg19.VGG19(weights='imagenet', include_top=True)

In [13]:
vgg19_naive.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [19]:
input_image = vgg19_naive.input

In [25]:
# from tensorflow.keras.layers import Flatten,Dropout,Dense
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Flatten, Dense, Dropout

# from tensorflow.keras.optimizers import SGD

x = vgg19_naive.get_layer('fc2').output
print(x.shape)
# vgg19_naive.add(Flatten())
# vgg19_naive.add(Dense(4096, activation='relu'))
# vgg19_naive.add(Dropout(0.5))
# vgg19_naive.add(Dense(4096, activation='relu'))
# vgg19_naive.add(Dropout(0.5))
# vgg19_naive.add(Dense(1000, activation='softmax'))

# x = layers.Conv1D(filters=None, kernel_size=1, activation='relu')(x)
# x = Flatten()(x)
# x = layers.Dense(4096, activation='relu')(x)
# x = layers.Conv1D(filters=4096, kernel_size=1, activation='relu')(x)
# print(x.shape)
# x = layers.MaxPooling2D(pool_size=(2, 2))(x)
# # x = Flatten()(x)
# x = Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(120, activation='softmax')(x)
print(x.shape)

(?, 4096)
(?, 120)


In [26]:
model = models.Model(inputs=input_image, outputs=x)
for layer in vgg19_naive.layers:
    layer.trainable = False

model.compile(
    optimizer='SGD', 
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [27]:
# Callbacks
class SaveModelandEval(callbacks.Callback):
    prev_res = list()
    def __init__(self):
        self.prev_res = [0., 0.]
    def on_epoch_end(self, epoch, logs):
        if epoch == 0: return
        if epoch % 5 == 0:
            res = model.evaluate_generator(Test, verbose=0, steps=None, workers=4, use_multiprocessing=True, max_queue_size=12)
            print('\n', res)
            if res[1] > self.prev_res[1]:
                models.save_model(model, 'vgg19_dfc120_%s_%s'%(epoch, int(res[1]*10000)) )
                self.prev_res = res

In [75]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [28]:
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import SGD

In [11]:
model = models.load_model('vgg19_dfc120_40_6751')

In [32]:
history_vgg19=model.fit_generator(
    Train, 
    steps_per_epoch=80, 
    epochs=101, 
    verbose=1, 
    workers=4,
    use_multiprocessing=True,
    max_queue_size=20,
    initial_epoch=0,
#     validation_data=Test,
#     validation_steps=20,
    callbacks=[SaveModelandEval()]
)

Epoch 1/101
80/80 [==============================] - 21s 257ms/step - loss: 2.8131 - acc: 0.4223
Epoch 2/101
80/80 [==============================] - 20s 251ms/step - loss: 2.8256 - acc: 0.4023
Epoch 3/101
80/80 [==============================] - 20s 252ms/step - loss: 2.7468 - acc: 0.4164
Epoch 4/101
80/80 [==============================] - 20s 251ms/step - loss: 2.6021 - acc: 0.4410
Epoch 5/101
80/80 [==============================] - 21s 260ms/step - loss: 2.7336 - acc: 0.4129
Epoch 6/101
79/80 [============================>.] - ETA: 0s - loss: 2.4894 - acc: 0.4498
 [1.1161121639302145, 0.7044289044289044]
80/80 [==============================] - 89s 1s/step - loss: 2.4884 - acc: 0.4488
Epoch 7/101
80/80 [==============================] - 20s 252ms/step - loss: 2.4072 - acc: 0.4570
Epoch 8/101
80/80 [==============================] - 20s 252ms/step - loss: 2.4449 - acc: 0.4520
Epoch 9/101
80/80 [==============================] - 20s 253ms/step - loss: 2.5053 - acc: 0.4438
Epoch 10/1

80/80 [==============================] - 20s 252ms/step - loss: 1.8140 - acc: 0.5504
Epoch 70/101
80/80 [==============================] - 20s 252ms/step - loss: 1.7471 - acc: 0.5613
Epoch 71/101
79/80 [============================>.] - ETA: 0s - loss: 1.7379 - acc: 0.5617
 [1.0685623348592386, 0.732983682983683]
80/80 [==============================] - 89s 1s/step - loss: 1.7331 - acc: 0.5637
Epoch 72/101
80/80 [==============================] - 20s 252ms/step - loss: 1.6774 - acc: 0.5660
Epoch 73/101
80/80 [==============================] - 20s 252ms/step - loss: 1.7117 - acc: 0.5629
Epoch 74/101
80/80 [==============================] - 20s 252ms/step - loss: 1.7459 - acc: 0.5473
Epoch 75/101
80/80 [==============================] - 20s 253ms/step - loss: 1.6980 - acc: 0.5766
Epoch 76/101
79/80 [============================>.] - ETA: 0s - loss: 1.6002 - acc: 0.5791
 [1.028884679842485, 0.7353146853146854]
80/80 [==============================] - 89s 1s/step - loss: 1.5985 - acc: 0.57

In [15]:
import json

with open('history_vgg19.json', 'w') as f:
    json.dump(history_vgg19.history, f)

# Fine Tune

In [33]:
model = models.load_model('vgg19_dfc120_30_7376')

In [17]:
for i, layer in enumerate(model.layers):
    print(i, layer.name)

(0, u'input_1')
(1, u'block1_conv1')
(2, u'block1_conv2')
(3, u'block1_pool')
(4, u'block2_conv1')
(5, u'block2_conv2')
(6, u'block2_pool')
(7, u'block3_conv1')
(8, u'block3_conv2')
(9, u'block3_conv3')
(10, u'block3_conv4')
(11, u'block3_pool')
(12, u'block4_conv1')
(13, u'block4_conv2')
(14, u'block4_conv3')
(15, u'block4_conv4')
(16, u'block4_pool')
(17, u'block5_conv1')
(18, u'block5_conv2')
(19, u'block5_conv3')
(20, u'block5_conv4')
(21, u'block5_pool')
(22, u'global_average_pooling2d')
(23, u'dropout')
(24, u'dense')


In [34]:
for layer in model.layers[:12]:
    layer.trainable = False
for layer in model.layers[12:]:
    layer.trainable = True

In [35]:
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import SGD

In [36]:
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [37]:
# Callbacks
class SaveModelandEval_finetune(callbacks.Callback):
    prev_res = list()
    def __init__(self):
        self.prev_res = [0., 0.]
    def on_epoch_end(self, epoch, logs):
        if epoch == 0: return
        if epoch % 3 == 0:
            res = model.evaluate_generator(Test, verbose=0, steps=None, workers=4, use_multiprocessing=True, max_queue_size=12)
            print('\n', res)
            if res[1] > self.prev_res[1]:
                models.save_model(model, 'vgg19_dfc120_finetune_%s_%s'%(epoch, int(res[1]*10000)) )
                self.prev_res = res

In [38]:
model.fit_generator(
    Train, 
    steps_per_epoch=80, 
    epochs=22, 
    verbose=1, 
    workers=4,
    use_multiprocessing=True,
    max_queue_size=20,
    initial_epoch=0,
    callbacks=[SaveModelandEval_finetune()]
)

Epoch 1/22
80/80 [==============================] - 37s 464ms/step - loss: 1.6655 - acc: 0.5527
Epoch 2/22
80/80 [==============================] - 34s 423ms/step - loss: 1.5787 - acc: 0.5547
Epoch 3/22
80/80 [==============================] - 34s 423ms/step - loss: 1.5828 - acc: 0.5563
Epoch 4/22
79/80 [============================>.] - ETA: 0s - loss: 1.4967 - acc: 0.5779
 [0.898190791537119, 0.7385780885780886]
80/80 [==============================] - 103s 1s/step - loss: 1.4912 - acc: 0.5793
Epoch 5/22
80/80 [==============================] - 34s 430ms/step - loss: 1.4623 - acc: 0.5891
Epoch 6/22
80/80 [==============================] - 34s 425ms/step - loss: 1.3301 - acc: 0.6180
Epoch 7/22
79/80 [============================>.] - ETA: 0s - loss: 1.3214 - acc: 0.6104
 [0.880990124828104, 0.742074592074592]
80/80 [==============================] - 103s 1s/step - loss: 1.3305 - acc: 0.6090
Epoch 8/22
80/80 [==============================] - 34s 425ms/step - loss: 1.3333 - acc: 0.6246

# Extract Xception bottleneck features

In [14]:
# INPUT_SIZE = 299
# POOLING = 'avg'
# x_train = np.zeros((len(labels), INPUT_SIZE, INPUT_SIZE, 3), dtype='float32')
# for i, img_id in tqdm(enumerate(labels['id'])):
#     img = read_img(img_id, 'train', (INPUT_SIZE, INPUT_SIZE))
#     x = xception.preprocess_input(np.expand_dims(img.copy(), axis=0))
#     x_train[i] = x
# print('Train Images shape: {} size: {:,}'.format(x_train.shape, x_train.size))

In [15]:
# Xtr_torch = torch . from_numpy (Xtr)
# Xv_torch = torch . from_numpy (Xv)
# print(Xv_torch.type)


In [16]:
# Xtr = x_train[train_idx]
# Xv = x_train[valid_idx]
# Xtr_torch = torch . from_numpy (Xtr)
# Xv_torch = torch . from_numpy (Xv)
# print((Xtr.shape, Xv.shape, ytr.shape, yv.shape))
# Xtr_torch=Xtr_torch.cuda()
# Xv_torch=Xv_torch.cuda()
# xception_bottleneck = xception.Xception(weights='imagenet', include_top=False, pooling=POOLING)
# train_x_bf = xception_bottleneck.predict(Xtr, batch_size=32, verbose=1)
# valid_x_bf = xception_bottleneck.predict(Xv, batch_size=32, verbose=1)
# print('Xception train bottleneck features shape: {} size: {:,}'.format(train_x_bf.shape, train_x_bf.size))
# print('Xception valid bottleneck features shape: {} size: {:,}'.format(valid_x_bf.shape, valid_x_bf.size))

In [17]:
# xception_bottleneck = xception.Xception(weights='imagenet', include_top=False, pooling=POOLING)

In [18]:
# Xtr_torch=torch . from_numpy (Xtr)

In [19]:
# train_x_bf = xception_bottleneck.predict(Xtr_torch, batch_size=32, verbose=1)

In [20]:
# xception_bottleneck = xception.Xception(weights='imagenet', include_top=False, pooling=POOLING)
# train_x_bf = xception_bottleneck.predict(Xtr, batch_size=32, verbose=1)
# valid_x_bf = xception_bottleneck.predict(Xv, batch_size=32, verbose=1)
# print('Xception train bottleneck features shape: {} size: {:,}'.format(train_x_bf.shape, train_x_bf.size))
# print('Xception valid bottleneck features shape: {} size: {:,}'.format(valid_x_bf.shape, valid_x_bf.size))

In [21]:
# Xtr_torch=Xtr_torch.cuda()
# Xv_torch=Xv_torch.cuda()

In [22]:
# print(Xtr_torch.type)

In [23]:
# a=torch.randn(3,5)